In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Introduction: visualisation des données et mise en place des variables**

On asigne les données aux variables d'entrainement et de test. 

In [ ]:
train=pd.read_csv("/kaggle/input/dont-overfit-ii/train.csv")
test=pd.read_csv("/kaggle/input/dont-overfit-ii/test.csv")


On affiche les données de train

In [ ]:
train.head()

Ici on affiche dans un graphique train et test, pour voir la difference de quantité de donnée 

In [ ]:
import matplotlib.pyplot as plt

names = ('train.csv', 'test.csv')
y_position = range(len(names))
 
plt.bar(
    y_position, 
    (train.shape[0], test.shape[0]), 
    align='center', 
    alpha=0.8
)
plt.xticks(y_position, names)
plt.ylabel('Nombre') 
plt.title('Quantité de données ')
plt.show()

On créer les variables pour les modeles de machines learning

In [ ]:

X=train.drop(["target", "id"], axis=1)
y=train["target"]

On affiche les lignes avec seulement y=1

In [ ]:
y[y==1]

**Création des modeles de machine learning**

On importe toutes les librairies dont on a besoin 

In [ ]:

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

Creation des modeles de machine learning; certain de classification et d'autres de regression pour voir la difference lors de ce probleme 

In [ ]:
model_ridge = linear_model.Ridge()
model_lasso = linear_model.Lasso()
model_log_reg = linear_model.LogisticRegression(solver='liblinear')
model_reg_lin = linear_model.LinearRegression()
model_knc = KNeighborsClassifier(n_neighbors=20)
model_forest_reg = RandomForestRegressor(n_estimators = 10, random_state = 0)
model_tree = DecisionTreeClassifier(max_depth=3, random_state=1)
model_forest = RandomForestClassifier(n_estimators=15, max_depth=2, random_state=1)
model_neighbor = KNeighborsClassifier(n_neighbors=5)

models = [model_ridge, model_lasso, model_log_reg,model_reg_lin, model_knc,model_forest_reg,model_tree,model_forest,model_neighbor]

**Entrainement des modeles**

In [ ]:
model_ridge.fit(X,y)
model_lasso.fit(X,y)
model_log_reg.fit(X,y)
model_reg_lin.fit(X,y)
model_knc.fit(X,y)
model_forest_reg.fit(X,y)
model_tree.fit(X,y)
model_forest.fit(X,y)
model_neighbor.fit(X,y)

On regade ce que donne les predictions apres entrainement 

In [ ]:
model_log_reg.fit(X,y)
model_log_reg.predict(X)

In [ ]:
model_forest_reg.fit(X,y)
model_forest_reg.predict(X)

In [ ]:
model_forest.fit(X,y)
model_forest.predict_proba(X)[:,1]

**Evaluation des modèles**

On essaie d'evaluer les modèles avec confusion_matrix

On a des modèles qui retournent du binaire et des continues, confusion_matrix ne fonctionne qu'avec ceux retournant des continues. 
Cette evaluation ne convient pas pour tous nos modèles 

In [ ]:
from sklearn.metrics import confusion_matrix
model_a_evaluer = model_ridge
confusion_matrix(y, model_a_evaluer.predict(X))

In [ ]:
model_a_evaluer = model_lasso
confusion_matrix(y, model_a_evaluer.predict(X))

In [ ]:
model_a_evaluer = model_log_reg
confusion_matrix(y, model_a_evaluer.predict(X))

Pas d'erreur pour 250 données 

In [ ]:
model_a_evaluer = model_reg_lin
confusion_matrix(y, model_a_evaluer.predict(X))

In [ ]:
model_a_evaluer = model_knc
confusion_matrix(y, model_a_evaluer.predict(X))

Ici, le KNeighborsClassifier fait 67 erreurs sur 250

In [ ]:
model_a_evaluer = model_forest_reg
confusion_matrix(y, model_a_evaluer.predict(X))

In [ ]:
model_a_evaluer = model_tree
confusion_matrix(y, model_a_evaluer.predict(X))

Ici DecisionTreeClassifier fait 37 erreurs sur 250

In [ ]:
model_a_evaluer = model_forest
confusion_matrix(y, model_a_evaluer.predict(X))

Ici, RandomForestClassifier fait 65 erreurs sur 250

In [ ]:
model_a_evaluer = model_neighbor
confusion_matrix(y, model_a_evaluer.predict(X))

Ici, KNeighborsClassifier fait 58 erreurs sur 250

Meme si cela ne permet pas d'evaluer tous les modeles, cela nous donne un premier avis sur les modèles qui sont plus performanats que d'autres 

Ici on va maintement évaluer les modèles avec une cross validation, on creer donc une fonction permettant d'évaluer les scores de tous nos modeles. On lui demmande d'afficher la moyennes de nos scores ainsi que l'équart tye pour avoir une vue global de chaque modèle

In [ ]:
def cVal_scores(model):
    scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
    print('Moyenne des scores: ', np.mean(scores))
    print('Ecart-type des scores: ', np.std(scores))
    print('\n') # Saut de ligne pour plus de visibilité

La boucle for nous permet de tester chaque modèles un à un dans notre fonction précédemment crée, puis d'afficher les scores

In [ ]:
for model in models:
    print(model)
    cVal_scores(model)

On peut voir que le modèle le plus performant est la regression lineaire avec une moyenne des scores la plus haute (0,7) et un écart type minime de 0,04 (le deuxieme plus petit)

**Recherche des meilleurs paramètres**

Maintenant on passe à la recherches des meilleurs parametres pour la regression lineaire. Pour cela on va tester tous les differents paramètres possibles un à un puis on va retourner les meilleurs. 
On utilise la fonction de sklearn GridSearchCV pour cela.
Inconvegnant : ce test est long puisqu'on va tester tous les paramètres, ici 1600 fits

In [ ]:
from sklearn.model_selection import GridSearchCV
penalite = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga','newton-cg', 'sag','lbfgs']

param = dict(penalty=penalite,
             C=C,
             class_weight=class_weight,
             solver=solver)

grid = GridSearchCV(estimator=model_log_reg, param_grid=param, scoring='roc_auc', verbose=1, n_jobs=-1)
result = grid.fit(X,y)

print('Meilleur Score: ', result.best_score_)
print('Meilleurs Paramètres: ', result.best_params_)

Ici on a une erreur, mais on va tester notre model avec les valeurs retournés 

In [ ]:
model_log_reg = linear_model.LogisticRegression(C=0.1, class_weight={1:0.6, 0:0.4}, penalty='l1', solver='liblinear')
cVal_scores(model_log_reg)

On refait la meme chose mais avec moins de valeurs à tester dans le solver, pour voir s'il y a une difference 
ici 640 fits, ce qui est beaucoup moins que la première fois, donc pour plus court à tester 

In [ ]:

penalite = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param = dict(penalty=penalite,
            C=C,
            class_weight=class_weight,
            solver=solver)

grid = GridSearchCV(estimator=model_log_reg, param_grid=param, scoring='roc_auc', verbose=1, n_jobs=-1)
result = grid.fit(X,y)

print('Meilleur Score: ', result.best_score_)
print('Meilleurs Paramètres: ', result.best_params_)

Il est intéressant de voir qu'on a plus d'erreurs, et une difference au niveau des parametres retournés


In [ ]:
model_log_reg = linear_model.LogisticRegression(C=0.1, class_weight={1:0.6, 0:0.4}, penalty='l1', solver='liblinear')
cVal_scores(model_log_reg)

Enfin, on cherche notre target soit Y_test, avec les donnée de test et les meilleurs parametres apres avoir entrainé le modèle 

In [ ]:
X_test=test.drop(["id"], axis=1)
model_log_reg.fit(X,y),

y_test=model_log_reg.predict(X_test)

y_test



In [ ]:
y_test[y_test==0]

In [ ]:
y_test[y_test==1]